In [1]:
from pathlib import Path
import pandas as pd
import tszip

In [2]:
data_dir = Path("../data")

In [3]:
ts_file = data_dir / "v1-beta1_2023-02-21.pp.md.bpshift.ts.dated.il.tsz"
ts = tszip.decompress(ts_file)
ts

In [4]:
#matches_file = data_dir / "v1-beta1.matches.tsv"
#matches = pd.read_csv(matches_file, sep="\t")
#matches.head(1)

In [5]:
metadata_file = data_dir / "run_metadata.v04.tsv.gz"
metadata = pd.read_csv(metadata_file, sep="\t")
metadata.head(1)

/var/folders/4h/9ylb643n37d8jc7srjm7qzk00000gr/T/ipykernel_64920/3676841558.py:2: DtypeWarning: Columns (16,18,19,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metadata_file, sep="\t")


,Study,Sample,Experiment,Run,Platform,Country,Region,Collection_date,First_created,Date_tree,...,Viridian_amplicon_scheme,Viridian_N,Genbank_N,Viridian_pangolin,Viridian_scorpio,Genbank_pangolin,Genbank_scorpio,Genbank_tree_name,Viridian_cons_len,Viridian_cons_het
0,PRJEB47121,SAMEA9781395,ERX6172603,ERR6546375,ILLUMINA,Estonia,none,2021-08-02,2021-08-22,2021-08-02,...,COVID-ARTIC-V3,306,.,AY.100,Delta (B.1.617.2-like),.,.,.,29810,0


In [6]:
from collections import defaultdict
pangos = defaultdict(int)
for p in metadata["Viridian_pangolin"]:
    if p.startswith("X"):
        # Lumps together X?.*
        pangos[p.split(".")[0]] += 1
pango_df = pd.DataFrame(
    [{"pango": k} for k in pangos.keys()]
).sort_values(by="pango").reset_index(drop=True)
pango_df.head(1)

,pango
0,XA


In [7]:
# Get sample counts in Viridian v04.
data = {}
for p in pango_df["pango"]:
    data[p] = {
        "viridian04_num_samples_pass": 0,
        "viridian04_num_samples_pass_tree": 0,
        "viridian04_num_samples_pass_tree_dated": 0,
        "viridian04_num_samples_pass_tree_dated_notnye": 0,
    }
for i, row in metadata.iterrows():
    p = row["Viridian_pangolin"].split(".")[0]
    if p.startswith("X"):
        if row["Viridian_result"] == "PASS":
            data[p]["viridian04_num_samples_pass"] += 1
            if row["In_Viridian_tree"] == "T":
                data[p]["viridian04_num_samples_pass_tree"] += 1
                if row["Date_tree"] != ".":
                    data[p]["viridian04_num_samples_pass_tree_dated"] += 1
                    if row["Date_tree"] != "2020-12-31":
                        data[p]["viridian04_num_samples_pass_tree_dated_notnye"] += 1

In [8]:
viridian04_df = pd.DataFrame([{"pango": k} | v for k, v in data.items()])
viridian04_df.head(1)

,pango,viridian04_num_samples_pass,viridian04_num_samples_pass_tree,viridian04_num_samples_pass_tree_dated,viridian04_num_samples_pass_tree_dated_notnye
0,XA,43,42,42,42


In [9]:
def _rename_columns(df, rsuffix):
    return dict(zip(
        df.columns,
        [
            (lambda x: x if x == "pango" else x + rsuffix)(x)
            for x in df.columns
        ]
    ))

In [10]:
pangodesig_file = data_dir / "bigtable_pangodesig.csv"
pangodesig_df = pd.read_csv(pangodesig_file)
pangodesig_df = pangodesig_df.rename(
    columns=_rename_columns(df=pangodesig_df, rsuffix="_pangodesig")
)
pangodesig_df.head(1)

,pango,github_issue_pangodesig,type_pangodesig,parent_left_pango_pangodesig,parent_right_pango_pangodesig,parents_extra_pangodesig,interval_left_pangodesig,interval_right_pangodesig,intervals_extra_pangodesig
0,XA,https://github.com/cov-lineages/pango-designat...,simple,B.1.177,B.1.1.7,NaN,21254,21765,NaN


In [11]:
rhgisaid_file = data_dir / "bigtable_rhgisaid.csv"
rhgisaid_df = pd.read_csv(rhgisaid_file)
rhgisaid_df = rhgisaid_df.rename(
    columns=_rename_columns(df=rhgisaid_df, rsuffix="_rhgisaid")
)
rhgisaid_df.head(1)

,pango,is_in_results_rhgisaid,type_rhgisaid,parent_left_pango_rhgisaid,parent_right_pango_rhgisaid,parents_extra_rhgisaid,interval_left_rhgisaid,interval_right_rhgisaid,intervals_extra_rhgisaid
0,XA,True,simple,B.1.177.18,B.1.1.7,NaN,21257.0,21258.0,NaN


In [12]:
rhnextstrain_file = data_dir / "bigtable_rhnextstrain.csv"
rhnextstrain_df = pd.read_csv(rhnextstrain_file)
rhnextstrain_df = rhnextstrain_df.rename(
    columns=_rename_columns(df=rhnextstrain_df, rsuffix="_rhnextstrain")
)
rhnextstrain_df.head(1)

,pango,is_in_results_rhnextstrain,type_rhnextstrain,parent_left_pango_rhnextstrain,parent_right_pango_rhnextstrain,parents_extra_rhnextstrain,interval_left_rhnextstrain,interval_right_rhnextstrain,intervals_extra_rhnextstrain
0,XA,True,simple,B.1.177.18,B.1.1.7,NaN,21300.0,21301.0,NaN


In [13]:
%run parse_covrecomb_results.ipynb

In [14]:
covrecomb_file = data_dir / "bigtable_covrecomb.csv"
covrecomb_df = pd.read_csv(covrecomb_file)
covrecomb_df = covrecomb_df.rename(
    columns=_rename_columns(df=covrecomb_df, rsuffix="_covrecomb")
)
covrecomb_df.head(1)

,pango,is_in_results_covrecomb,type_covrecomb,parent_left_pango_covrecomb,parent_right_pango_covrecomb,interval_left_covrecomb,interval_right_covrecomb
0,XA,True,simple,B.1.177.18,B.1.1.7*,21255.0,21765.0


In [15]:
sc2ts_file = data_dir / "bigtable_sc2ts.csv"
sc2ts_df = pd.read_csv(sc2ts_file)
sc2ts_df = sc2ts_df.rename(
    columns=_rename_columns(df=sc2ts_df, rsuffix="_sc2ts")
)
sc2ts_df.head(1)

,pango,github_issue_sc2ts,is_in_results_sc2ts,is_in_arg_sc2ts,is_in_matches_sc2ts,is_clean_sc2ts,is_nested_sc2ts,re_node_sc2ts,type_sc2ts,parent_left_pango_sc2ts,parent_right_pango_sc2ts,parents_extra_sc2ts,interval_left_sc2ts,interval_right_sc2ts,intervals_extra_sc2ts
0,XA,https://github.com/jeromekelleher/sc2ts-paper/...,True,True,NaN,True,False,122444.0,simple,B.1.177.18,B.1.1.7,NaN,20411,21767,NaN


In [16]:
recomb_file = data_dir / "recombinants.csv"
recomb_df = pd.read_csv(recomb_file)
for i, row in sc2ts_df.iterrows():
    if (row["re_node_sc2ts"] != float("nan")) and (row["re_node_sc2ts"] != -1):
        if any(recomb_df["recombinant"] == row["re_node_sc2ts"]):
            tmp_df = recomb_df[recomb_df["recombinant"] == row["re_node_sc2ts"]].reset_index(drop=True)
            assert tmp_df["parent_left_pango"][0] == row["parent_left_pango_sc2ts"]
            assert tmp_df["parent_right_pango"][0] == row["parent_right_pango_sc2ts"]
            assert tmp_df["interval_left"][0] == int(row["interval_left_sc2ts"])
            assert tmp_df["interval_right"][0] == int(row["interval_right_sc2ts"])

In [17]:
merged_df = pango_df\
    .merge(viridian04_df, on="pango")\
    .merge(pangodesig_df, on="pango")\
    .merge(sc2ts_df, on="pango")\
    .merge(rhgisaid_df, on="pango")\
    .merge(rhnextstrain_df, on="pango")\
    .merge(covrecomb_df, on="pango")
merged_df.columns

Index(['pango', 'viridian04_num_samples_pass',
       'viridian04_num_samples_pass_tree',
       'viridian04_num_samples_pass_tree_dated',
       'viridian04_num_samples_pass_tree_dated_notnye',
       'github_issue_pangodesig', 'type_pangodesig',
       'parent_left_pango_pangodesig', 'parent_right_pango_pangodesig',
       'parents_extra_pangodesig', 'interval_left_pangodesig',
       'interval_right_pangodesig', 'intervals_extra_pangodesig',
       'github_issue_sc2ts', 'is_in_results_sc2ts', 'is_in_arg_sc2ts',
       'is_in_matches_sc2ts', 'is_clean_sc2ts', 'is_nested_sc2ts',
       're_node_sc2ts', 'type_sc2ts', 'parent_left_pango_sc2ts',
       'parent_right_pango_sc2ts', 'parents_extra_sc2ts',
       'interval_left_sc2ts', 'interval_right_sc2ts', 'intervals_extra_sc2ts',
       'is_in_results_rhgisaid', 'type_rhgisaid', 'parent_left_pango_rhgisaid',
       'parent_right_pango_rhgisaid', 'parents_extra_rhgisaid',
       'interval_left_rhgisaid', 'interval_right_rhgisaid',
       

In [ ]:
merged_file = data_dir / "bigtable_merged.csv"
merged_df.to_csv(merged_file, index=False)